# Find distribution of cel/pill variants

In [2]:
# Load processed data (necessary?)
import pandas as pd

path = '../../data/incels/processed_comments.pkl'
data = pd.read_pickle(path)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6248230 entries, 0 to 6248229
Data columns (total 11 columns):
 #   Column                            Dtype         
---  ------                            -----         
 0   type                              object        
 1   forum                             object        
 2   thread                            object        
 3   username                          object        
 4   date                              object        
 5   content                           object        
 6   parsed_date                       datetime64[ns]
 7   content_orig                      object        
 8   netmapper_identity_matches        object        
 9   netmapper_identity_matches_spans  object        
 10  actions_attributes                object        
dtypes: datetime64[ns](1), object(10)
memory usage: 524.4+ MB


In [5]:
# Load incels vocab and term counts from Gensim model
from gensim.models import Word2Vec

path = '../models/emb/incels.model'
model = Word2Vec.load(path)

In [15]:
from collections import Counter

usernames = ['animecel2d', '@aaaaaaaaaaacel', 'tehgymcel420', '@tehgymcel420', '@diocel', 'legendarywristcel', 'aaaaaaaaaaacel', 'itsover4cel',
             '@ritalincel', 'gymletethnicel', 'incelinside', 'sillytruecel', '@cowcel', 'cowcel', 'gyros_pretcel', 'ethnicelnl', '@zettacel',
            'zettacel', 'thebasedcel', '@thoughtfulcel', 'thoughtfulcel', 'thereaclchincel', 'turbocuckcel_7000', 'supersaiyangymcel', 'alt="cowcel"]https://incels.co',
            '@sergeantincel', '@gymletethnicel', 'ppecel', '@ppecel', 'finncel', '@finncel', 'speedtypingincel', 'itsogrecel', 'dogcel256',
            'idlevillagercel', 'framecel222', 'crestfallencel', 'templarcel421', '@eskimocel', 'eskimocel', 'cameroncel',]
exclude = usernames + [
    'incels', 'incel', 'incels.is', 'inceldom', 'celibate', 'celebrate', 'celebrity', 'celebrities', 'inceltears', 'cells', 'excel', 'excels',
    'excellent', 'nicely',
]
cel_variants = Counter({wd: model.wv.get_vecattr(wd, 'count') for wd in model.wv.key_to_index.keys() if 'cel' in wd and not wd in exclude})
cel_variants.most_common(100)

[('fakecel', 36499),
 ('volcel', 35796),
 ('incels.co', 32872),
 ('diocel', 31577),
 ('truecel', 29972),
 ('truecels', 20009),
 ('fakecels', 16226),
 ('ritalincel', 14562),
 ('trucel', 13835),
 ('mentalcel', 12992),
 ('cel', 12768),
 ('ricecels', 11437),
 ('gymcelled', 11044),
 ('lookscel', 11030),
 ('greycel', 11018),
 ('cels', 9614),
 ('daydreamincel', 9109),
 ('ricecel', 9107),
 ('@animecel2d', 8629),
 ('incellectual', 8567),
 ('manicel', 8567),
 ('gymcel', 8517),
 ('brocel', 7766),
 ('currycel', 7698),
 ('rightfulcel', 7510),
 ('gymcelling', 7173),
 ('braincels', 6569),
 ('acnescarcel', 6476),
 ('angrycurrycel', 6449),
 ('volcels', 6305),
 ('graycel', 6044),
 ('aedracel', 6041),
 ('blackcel', 5904),
 ('mentalcels', 5868),
 ('oldcel', 5866),
 ('copinggymcel', 5615),
 ('escortcel', 5545),
 ('caesercel', 5477),
 ('deformaspergercel', 5458),
 ('josefmengelecel', 5423),
 ('currycels', 5366),
 ('incelcream', 4911),
 ('femcels', 4901),
 ('jockcel', 4885),
 ('whitecels', 4745),
 ('johnwick